In [36]:
import sys
sys.path.append('../src/')

In [37]:
import pandas as pd
import numpy as np

import math
import matplotlib.pyplot as plt
import seaborn as sns

import os.path as osp
import os

In [65]:
pwd

In [66]:
data = pd.read_csv('/Users/fuyuchuan/downloads/res_0416.csv')
data

In [ ]:
plt.plot()

In [71]:
list(data.loc[0, :])

In [46]:
file_path = '/Users/fuyuchuan/thesis/code/eval_lp_xai/outputs/cora/gcn/zorro_baseline/metric_list.npy'
res = np.load(file_path, allow_pickle=True)

In [62]:
type(res[20]['node_sparsity_original'])

# Without Ground Truth

## 1. Fidelity, feature and node sparsity

In [84]:
def calculate_mean_std(metric):
    return np.round(np.mean(metric), 4), np.round(np.std(metric), 4)

def load_metrics(dataset, model, explainer):
    file_path = '/Users/fuyuchuan/thesis/code/eval_lp_xai/outputs/{}/{}/{}/metric_list.npy'.format(dataset, model, explainer)
    res = np.load(file_path, allow_pickle=True)
    
    # fidelity
    fidelity = np.array([r['fidelity'] for r in res])
    fidelity_change_in_pred_prob = np.array([r['fidelity_change_in_pred_prob'] for r in res])
    fid_plus_label = np.array([r['fidelity_original']['fid_plus_label'] for r in res])
    fid_plus_prob = np.array([r['fidelity_original']['fid_plus_prob'] for r in res])
    fid_minus_label = np.array([r['fidelity_original']['fid_minus_label'] for r in res])
    fid_minus_prob = np.array([r['fidelity_original']['fid_minus_prob'] for r in res])

    # sparsity
    feature_sparsity = np.array([r['feature_sparsity'] for r in res])
    edge_sparsity = np.array([r['edge_sparsity'] for r in res])
    node_sparsity_converted = np.array([r['node_sparsity_converted'] for r in res])
    node_sparsity_original = np.array([r['node_sparsity_original'] for r in res])

    # # Remove nan elements and the fidelity and feature sparsity corresponding to nan values in node sparsity
    # fidelity_list = fidelity_list[~np.isnan(node_sparsity_list)]
    # # validity_list = validity_list[~np.isnan(node_sparsity_list)]
    # feature_sparsity_list = feature_sparsity_list[~np.isnan(node_sparsity_list)]
    # node_sparsity_list = node_sparsity_list[~np.isnan(node_sparsity_list)]

    print('fidelity: ', calculate_mean_std(fidelity))
    print('fidelity_change_in_pred_prob: ', calculate_mean_std(fidelity_change_in_pred_prob))
    print('fid_plus_label: ', calculate_mean_std(fid_plus_label))
    print('fid_plus_prob: ', calculate_mean_std(fid_plus_prob))
    print('fid_minus_label: ', calculate_mean_std(fid_minus_label))
    print('fid_minus_prob: ', calculate_mean_std(fid_minus_prob))
    
    
    if np.isnan([r['feature_sparsity'] for r in res]).any():
        cnt = len(feature_sparsity)
        feature_sparsity = feature_sparsity[~np.isnan(feature_sparsity)]
        print(f'  Filtered {cnt - len(feature_sparsity)} nan samples in feature_sparsity.')
    print('feature_sparsity: ', calculate_mean_std(feature_sparsity))
    
    if explainer in ['gnnexplainer', 'ig', 'deconvolution']:
        print('edge_sparsity: ', calculate_mean_std(edge_sparsity))
        print('node_sparsity_converted: ', calculate_mean_std(node_sparsity_converted))
    
    if np.isnan([r['node_sparsity_original'] for r in res]).any():
        cnt = len(node_sparsity_original)
        node_sparsity_original = node_sparsity_original[~np.isnan(node_sparsity_original)]
        print(f'  Filtered {cnt - len(node_sparsity_original)} nan samples in node_sparsity.')
        
    print('node_sparsity_original: ', calculate_mean_std(node_sparsity_original))
    print('total_time_to_explain: ', res[-1]['total_time_to_explain'])
    
    if explainer == 'zorro':
        print('count: ')
        print([r['count'] for r in res])

    return fidelity


def load_fidelity_validity(dataset, model, explainer):
    file_path = '/Users/fuyuchuan/thesis/code/eval_lp_xai/outputs/{}/{}/{}/metric_list.npy'.format(dataset, model, explainer)
    res = np.load(file_path, allow_pickle=True)
    fidelity_list = np.array([r['fidelity'] for r in res])
    validity_list = np.array([r['validity'] for r in res])

    print('fidelity: ', calculate_mean_std(fidelity_list))
    print('validity: ', calculate_mean_std(validity_list))

    return fidelity_list, validity_list

def load_pgmexplainer(dataset, model, explainer):
    file_path = '/Users/fuyuchuan/thesis/code/eval_lp_xai/outputs/{}/{}/{}/metric_list.npy'.format(dataset, model, explainer)
    res = np.load(file_path, allow_pickle=True)
    fidelity = np.array([r['fidelity'] for r in res])
    node_sparsity_original = np.array([r['node_sparsity_original'] for r in res])
#     print(node_sparsity_original)
    node_sparsity_original = node_sparsity_original[~np.isnan(node_sparsity_original)]
    
    print('fidelity: ', calculate_mean_std(fidelity))
    print('node_sparsity_original: ', calculate_mean_std(node_sparsity_original))

    return fidelity, node_sparsity_original

def load_reduced_complete_exp(dataset, model, explainer):
    file_path = '/Users/fuyuchuan/thesis/code/eval_lp_xai/outputs/{}/{}/{}/metric_list.npy'.format(dataset, model, explainer)
    res = np.load(file_path, allow_pickle=True)
    fidelity = np.array([r['fidelity'] for r in res])
    node_sparsity_original = np.array([r['node_sparsity_original'] for r in res])
#     print(node_sparsity_original)
    node_sparsity_original = node_sparsity_original[~np.isnan(node_sparsity_original)]

    print('fidelity: ', calculate_mean_std(fidelity))
    print('node_sparsity_original: ', calculate_mean_std(node_sparsity_original))

    return fidelity, node_sparsity_original


def load_zorro_baseline(dataset, model, explainer):
    file_path = '/Users/fuyuchuan/thesis/code/eval_lp_xai/outputs/{}/{}/{}/metric_list.npy'.format(dataset, model, explainer)
    res = np.load(file_path, allow_pickle=True)
    fidelity = np.array([r['fidelity'] for r in res])
    
    node_sparsity_original = np.array([r['node_sparsity_original'] for r in res])
#     print(node_sparsity_original)
    node_sparsity_original = node_sparsity_original[~np.isnan(node_sparsity_original)]
    
    feature_sparsity = np.array([r['feature_sparsity'] for r in res])
    feature_sparsity = feature_sparsity[~np.isnan(feature_sparsity)]

    print('fidelity: ', calculate_mean_std(fidelity))
    print('node_sparsity_original: ', calculate_mean_std(node_sparsity_original))
    print('feature_sparsity: ', calculate_mean_std(feature_sparsity))

    return fidelity, node_sparsity_original, feature_sparsity


def load_reduced_complete_exp_original_fidelity(dataset, model, explainer):
    file_path = '/Users/fuyuchuan/thesis/code/eval_lp_xai/outputs/{}/{}/{}/fidelity_original_list.npy'.format(dataset, model, explainer)
    res = np.load(file_path, allow_pickle=True)
    
    fid_plus_label = np.array([r['fid_plus_label'] for r in res])
    fid_plus_prob = np.array([r['fid_plus_prob'] for r in res])
    fid_minus_label = np.array([r['fid_minus_label'] for r in res])
    fid_minus_prob = np.array([r['fid_minus_prob'] for r in res])
    
    print('fid_plus_label: ', calculate_mean_std(fid_plus_label))
    print('fid_plus_prob: ', calculate_mean_std(fid_plus_prob))
    print('fid_minus_label: ', calculate_mean_std(fid_minus_label))
    print('fid_minus_prob: ', calculate_mean_std(fid_minus_prob))
    
    return fid_plus_label, fid_plus_prob, fid_minus_label, fid_minus_prob

In [39]:
load_metrics('cora', 'gcn', 'gnnexplainer')

In [40]:
load_metrics('cora', 'gcn', 'ig')

In [41]:
load_metrics('cora', 'gcn', 'deconvolution')

In [42]:
load_metrics('cora', 'gcn', 'pgmexplainer')

In [89]:
load_metrics('cora', 'gcn', 'softzorro')

In [79]:
# tau: 0.1
load_metrics('cora', 'gcn', 'zorro_baseline')

In [86]:
# tau: 0.3, option 1
load_metrics('cora', 'gcn', 'zorro')

In [87]:
# tau: 0.3, option 3
load_metrics('cora', 'gcn', 'zorro')

In [88]:
# tau: 0.1, option 3
load_metrics('cora', 'gcn', 'zorro')

In [97]:
count_list = [23, 31, 40, 3, 7, 85, 4, 16, 145, 12, 5, 2, 90, 12, 11, 15, 131, 54, 9, 121, 0, 22, 8, 71, 15, 10, 29, 6, 82, 138, 6, 174, 13, 134, 28, 3, 8, 7, 0, 130, 126, 2, 115, 8, 9, 3, 3, 5, 6, 68, 7, 10, 3, 3, 9, 148, 9, 3, 84, 21, 6, 122, 36, 42, 6, 24, 0, 45, 35, 4, 3, 6, 85, 4, 13, 5, 8, 0, 115, 136, 16, 34, 44, 4, 163, 7, 163, 159, 3, 127, 28, 3, 17]
np.mean(count_list)

In [99]:
np.median(count_list)

In [98]:
# set the threshold to be 100.
plt.hist(count_list, rwidth=0.8)

In [101]:
# tau: 0.1, option 3, max 100 counts
load_metrics('cora', 'gcn', 'zorro')

In [103]:
# tau: 0.1, option 3, max 50 counts
load_metrics('cora', 'gcn', 'zorro')

In [104]:
# tau: 0.1, option 3, max 75 counts
load_metrics('cora', 'gcn', 'zorro')

In [105]:
count_list = [23, 31, 40, 3, 7, 75, 4, 16, 75, 12, 5, 2, 75, 12, 11, 15, 75, 54, 9, 75, 0, 22, 8, 71, 15, 10, 29, 6, 75, 75, 6, 75, 13, 75, 28, 3, 8, 7, 0, 75, 75, 2, 75, 8, 9, 3, 3, 5, 6, 68, 7, 10, 3, 3, 9, 75, 9, 3, 75, 21, 6, 75, 36, 42, 6, 24, 0, 45, 35, 4, 3, 6, 75, 4, 13, 5, 8, 0, 75, 75, 16, 34, 44, 4, 75, 7, 75, 75, 3, 75, 28, 3, 17]
plt.hist(count_list, rwidth=0.8)

In [106]:
# tau: 0.1, option 4
load_metrics('cora', 'gcn', 'zorro')

In [107]:
count_list = [23, 24, 40, 3, 7, 102, 4, 15, 152, 10, 4, 2, 52, 11, 10, 5, 82, 36, 9, 75, 0, 104, 7, 208, 14, 8, 21, 6, 79, 155, 6, 152, 9, 125, 18, 3, 5, 6, 0, 97, 153, 2, 42, 8, 9, 3, 3, 4, 6, 56, 6, 9, 3, 3, 7, 154, 8, 3, 81, 20, 5, 88, 64, 42, 6, 24, 0, 44, 79, 4, 3, 6, 72, 4, 13, 3, 7, 0, 115, 96, 15, 33, 47, 4, 147, 6, 69, 158, 3, 191, 18, 3, 16]
plt.hist(count_list, rwidth=0.8)

In [108]:
# tau: 0.1, option 5
load_metrics('cora', 'gcn', 'zorro')

In [109]:
count_list = [23, 24, 65, 3, 7, 102, 5, 11, 84, 10, 6, 2, 52, 15, 10, 5, 82, 36, 9, 156, 0, 104, 7, 208, 19, 8, 21, 6, 87, 102, 6, 155, 14, 121, 18, 3, 5, 12, 0, 96, 153, 2, 100, 8, 10, 3, 3, 4, 7, 60, 6, 9, 3, 3, 7, 154, 9, 3, 60, 20, 5, 111, 64, 68, 6, 34, 0, 48, 79, 4, 3, 6, 89, 4, 13, 3, 7, 0, 115, 96, 15, 32, 69, 4, 130, 6, 48, 159, 3, 191, 15, 3, 18]
plt.hist(count_list, rwidth=0.8)

In [110]:
# tau: 0.05, option 6
load_metrics('cora', 'gcn', 'zorro')

In [111]:
count_list = [28, 100, 49, 4, 9, 100, 7, 35, 100, 27, 13, 3, 87, 21, 22, 84, 100, 61, 11, 46, 2, 48, 8, 50, 22, 12, 25, 13, 100, 100, 7, 100, 33, 100, 33, 4, 11, 19, 0, 94, 100, 1, 100, 13, 14, 6, 4, 11, 8, 53, 8, 26, 5, 3, 16, 100, 13, 4, 100, 68, 7, 100, 100, 100, 8, 37, 3, 75, 100, 21, 4, 15, 100, 5, 33, 65, 10, 2, 100, 100, 28, 39, 76, 5, 100, 67, 58, 100, 4, 100, 30, 3, 38]
plt.hist(count_list, rwidth=0.8)

In [112]:
# tau: 0.02, option 6
load_metrics('cora', 'gcn', 'zorro')

In [47]:
fidelity, node_sparsity_original = load_reduced_complete_exp('cora', 'gcn', 'reduced_graph_complete_explanation')
print('Num of nodes: ', len(fidelity))

In [48]:
fid_plus_label, fid_plus_prob, fid_minus_label, fid_minus_prob = \
    load_reduced_complete_exp_original_fidelity('cora', 'gcn', 'reduced_graph_complete_explanation')

In [30]:
fid_minus_label

### reduced_graph_complete_explanation

Commpute the upper limit of RDT-Fidelity when the size of the computation graph is reduced.

#### Cora

In [101]:
# top_nodes: 50% of the largest number of neighbors for each end
fidelity, node_sparsity_original = load_reduced_complete_exp('cora', 'gcn', 'reduced_graph_complete_explanation')
print('Num of nodes: ', len(fidelity))

In [104]:
# top_nodes: 10 for each end
fidelity, node_sparsity_original = load_reduced_complete_exp('cora', 'gcn', 'reduced_graph_complete_explanation')
print('Num of nodes: ', len(fidelity))

In [105]:
# top_nodes: 9 for each end
fidelity, node_sparsity_original = load_reduced_complete_exp('cora', 'gcn', 'reduced_graph_complete_explanation')
print('Num of nodes: ', len(fidelity))

In [106]:
# top_nodes: 8 for each end
fidelity, node_sparsity_original = load_reduced_complete_exp('cora', 'gcn', 'reduced_graph_complete_explanation')
print('Num of nodes: ', len(fidelity))

In [107]:
# top_nodes: 7 for each end
fidelity, node_sparsity_original = load_reduced_complete_exp('cora', 'gcn', 'reduced_graph_complete_explanation')
print('Num of nodes: ', len(fidelity))

In [108]:
# top_nodes: 6 for each end
fidelity, node_sparsity_original = load_reduced_complete_exp('cora', 'gcn', 'reduced_graph_complete_explanation')
print('Num of nodes: ', len(fidelity))

In [109]:
# top_nodes: 5 for each end
fidelity, node_sparsity_original = load_reduced_complete_exp('cora', 'gcn', 'reduced_graph_complete_explanation')
print('Num of nodes: ', len(fidelity))

In [110]:
# top_nodes: 4 for each end
fidelity, node_sparsity_original = load_reduced_complete_exp('cora', 'gcn', 'reduced_graph_complete_explanation')
print('Num of nodes: ', len(fidelity))

In [111]:
# top_nodes: 3 for each end
fidelity, node_sparsity_original = load_reduced_complete_exp('cora', 'gcn', 'reduced_graph_complete_explanation')
print('Num of nodes: ', len(fidelity))

In [112]:
# top_nodes: 2 for each end
fidelity, node_sparsity_original = load_reduced_complete_exp('cora', 'gcn', 'reduced_graph_complete_explanation')
print('Num of nodes: ', len(fidelity))

In [113]:
# top_nodes: 1 for each end
fidelity, node_sparsity_original = load_reduced_complete_exp('cora', 'gcn', 'reduced_graph_complete_explanation')
print('Num of nodes: ', len(fidelity))

In [115]:
# top_nodes: 0 for each end
fidelity, node_sparsity_original = load_reduced_complete_exp('cora', 'gcn', 'reduced_graph_complete_explanation')
print('Num of nodes: ', len(fidelity))

#### PubMed

In [139]:
# top_nodes: 50% of the largest number of neighbors for each end
fidelity, node_sparsity_original = load_reduced_complete_exp('PubMed', 'gcn', 'reduced_graph_complete_explanation')
print('Num of nodes: ', len(fidelity))

In [141]:
# top_nodes: 20 for each end
fidelity, node_sparsity_original = load_reduced_complete_exp('PubMed', 'gcn', 'reduced_graph_complete_explanation')
print('Num of nodes: ', len(fidelity))

In [142]:
# top_nodes: 15 for each end
fidelity, node_sparsity_original = load_reduced_complete_exp('PubMed', 'gcn', 'reduced_graph_complete_explanation')
print('Num of nodes: ', len(fidelity))

In [140]:
# top_nodes: 10 for each end
fidelity, node_sparsity_original = load_reduced_complete_exp('PubMed', 'gcn', 'reduced_graph_complete_explanation')
print('Num of nodes: ', len(fidelity))

In [143]:
# top_nodes: 5 for each end
fidelity, node_sparsity_original = load_reduced_complete_exp('PubMed', 'gcn', 'reduced_graph_complete_explanation')
print('Num of nodes: ', len(fidelity))

### reduced_graph_complete_explanation, res of fidelity+ and fidelity-

1. The following results are based on top important neighbors.

Fidelity+
Mask the important nodes.

Cora: significantly decreases the probability of the existence of a link when important nodes are masked. It indicates these nodes are indeed important ones.


Fidelity-
Only use the important nodes.

Cora: only results in small changes in the probability, which means only using the important nodes can lead to the correct prediction.

2. Now replace the top nodes with random nodes in the computation subgraph.



#### Cora

In [153]:
fid_plus_label, fid_plus_prob, fid_minus_label, fid_minus_prob = \
    load_reduced_complete_exp_original_fidelity('cora', 'gcn', 'reduced_graph_complete_explanation')

#### PubMed

In [152]:
fid_plus_label, fid_plus_prob, fid_minus_label, fid_minus_prob = \
    load_reduced_complete_exp_original_fidelity('PubMed', 'gcn', 'reduced_graph_complete_explanation')

### Zorro

#### Add node and feature at the same time

In [78]:
# tau: 0.3
# time_taken for explaining:  2078.9753839969635
# time_taken for explaining + evaluation:  2088.6021041870117

fidelity, node_sparsity_original, feature_sparsity = load_zorro_baseline('cora', 'gcn', 'zorro')
print('Num of nodes: ', len(fidelity))

In [81]:
len(node_sparsity_original)

In [80]:
len(feature_sparsity)

In [84]:
# tau: 0.2
# time_taken for explaining:  4052.121970176697
# time_taken for explaining + evaluation:  4061.935608148575

fidelity, node_sparsity_original, feature_sparsity = load_zorro_baseline('cora', 'gcn', 'zorro')
print('Num of nodes: ', len(fidelity))

In [85]:
len(node_sparsity_original)

In [82]:
# tau: 0.15
# time_taken for explaining:  6054.12283039093
# time_taken for explaining + evaluation:  6063.590988874435

fidelity, node_sparsity_original, feature_sparsity = load_zorro_baseline('cora', 'gcn', 'zorro')
print('Num of nodes: ', len(fidelity))

In [83]:
len(node_sparsity_original)

#### Use node masks from zorro_baseline as fixed

In [88]:
# tau: 0.3
# time_taken for explaining:  978.5579869747162
# time_taken for explaining + evaluation:  987.9928789138794

fidelity, node_sparsity_original, feature_sparsity = load_zorro_baseline('cora', 'gcn', 'zorro')
print('Num of nodes: ', len(fidelity))

In [89]:
len(node_sparsity_original)

In [86]:
# tau: 0.2
# time_taken for explaining:  1878.4841783046722
# time_taken for explaining + evaluation:  1887.8399891853333

fidelity, node_sparsity_original, feature_sparsity = load_zorro_baseline('cora', 'gcn', 'zorro')
print('Num of nodes: ', len(fidelity))

In [87]:
len(node_sparsity_original)

In [91]:
# tau: 0.15
# time_taken for explaining:  2074.754957675934
# time_taken for explaining + evaluation:  2084.5078279972076

fidelity, node_sparsity_original, feature_sparsity = load_zorro_baseline('cora', 'gcn', 'zorro')
print('Num of nodes: ', len(fidelity))

In [92]:
len(node_sparsity_original)

In [93]:
# tau: 0.1
# time_taken for explaining:  1722.9277935028076
# time_taken for explaining + evaluation:  1732.1560640335083
fidelity, node_sparsity_original, feature_sparsity = load_zorro_baseline('cora', 'gcn', 'zorro')
print('Num of nodes: ', len(fidelity))

#### Use node masks from zorro_baseline as initial

##### Cora

In [97]:
# tau: 0.2
# time_taken for explaining:  2358.9623806476593
# time_taken for explaining + evaluation:  2368.470980167389

fidelity, node_sparsity_original, feature_sparsity = load_zorro_baseline('cora', 'gcn', 'zorro')
print('Num of nodes: ', len(fidelity))

In [98]:
len(node_sparsity_original)

In [95]:
# tau: 0.15
# time_taken for explaining:  3167.487534046173
# time_taken for explaining + evaluation:  3177.1116440296173

fidelity, node_sparsity_original, feature_sparsity = load_zorro_baseline('cora', 'gcn', 'zorro')
print('Num of nodes: ', len(fidelity))

In [96]:
len(node_sparsity_original)

In [99]:
# tau: 0.1
# time_taken for explaining:  3506.214375257492
# time_taken for explaining + evaluation:  3515.8612580299377

fidelity, node_sparsity_original, feature_sparsity = load_zorro_baseline('cora', 'gcn', 'zorro')
print('Num of nodes: ', len(fidelity))

In [158]:
iterations = [23, 31, 40, 3, 7, 85, 4, 16, 145, 12, 5, 2, 90, 12, 11, 15, 131, 54, 9, 121, 0, 22, 8, 71, 15, 10, 29, 6, 82, 138, 6, 174, 13, 134, 28, 3, 8, 7, 0, 130, 126, 2, 115, 8, 9, 3, 3, 5, 6, 68, 7, 10, 3, 3, 9, 148, 9, 3, 84, 21, 6, 122, 36, 42, 6, 24, 0, 45, 35, 4, 3, 6, 85, 4, 13, 5, 8, 0, 115, 136, 16, 34, 44, 4, 163, 7, 163, 159, 3, 127, 28, 3, 17]
nodes = [31, 17, 128, 6, 16, 13, 8, 26, 58, 11, 11, 8, 20, 12, 21, 20, 18, 13, 15, 36, 0, 42, 8, 167, 25, 12, 13, 8, 119, 27, 11, 91, 25, 37, 51, 6, 6, 20, 0, 81, 27, 3, 47, 6, 38, 8, 4, 5, 6, 160, 8, 21, 6, 19, 13, 22, 13, 5, 53, 19, 15, 43, 40, 88, 13, 94, 0, 37, 18, 4, 32, 6, 81, 13, 25, 7, 25, 0, 3, 32, 21, 40, 57, 32, 74, 20, 60, 37, 7, 95, 25, 6, 30]
nodes_percent = [1.0, 0.16666666666666666, 0.8311688311688312, 0.15789473684210525, 0.5333333333333333, 1.0, 0.36363636363636365, 0.65, 1.0, 1.0, 0.4583333333333333, 0.02318840579710145, 1.0, 0.23076923076923078, 0.2413793103448276, 0.1724137931034483, 1.0, 1.0, 0.6521739130434783, 1.0, 0.0, 1.0, 0.6153846153846154, 0.9881656804733728, 0.6410256410256411, 1.0, 0.8666666666666667, 0.34782608695652173, 1.0, 1.0, 0.8461538461538461, 1.0, 0.4807692307692308, 0.9487179487179487, 0.28651685393258425, 0.3, 0.5454545454545454, 0.5405405405405406, 0.0, 1.0, 1.0, 0.75, 1.0, 0.24, 0.7755102040816326, 0.11764705882352941, 0.05970149253731343, 0.5555555555555556, 0.35294117647058826, 0.975609756097561, 0.5, 0.9130434782608695, 0.2857142857142857, 0.055232558139534885, 0.8125, 1.0, 0.48148148148148145, 0.5, 1.0, 1.0, 0.42857142857142855, 0.9555555555555556, 1.0, 0.7213114754098361, 0.1780821917808219, 0.7704918032786885, 0.0, 0.5068493150684932, 1.0, 0.5, 0.20512820512820512, 1.0, 1.0, 0.1511627906976744, 0.16778523489932887, 0.1346153846153846, 0.16025641025641027, 0.0, 1.0, 1.0, 0.1044776119402985, 1.0, 0.9661016949152542, 0.3333333333333333, 0.9736842105263158, 0.13333333333333333, 1.0, 1.0, 0.21875, 1.0, 1.0, 0.02631578947368421, 0.46875]
features = [160, 260, 101, 20, 41, 791, 21, 61, 1190, 70, 10, 10, 850, 80, 51, 121, 1260, 491, 51, 1041, 0, 61, 50, 90, 71, 50, 260, 30, 371, 1281, 30, 1300, 51, 1240, 110, 1, 51, 41, 0, 1071, 1151, 1, 971, 60, 41, 20, 20, 30, 31, 51, 41, 21, 20, 11, 40, 1391, 50, 11, 611, 121, 31, 1021, 170, 101, 30, 31, 0, 301, 280, 30, 11, 41, 681, 20, 100, 31, 61, 0, 1131, 1220, 110, 171, 190, 21, 1280, 60, 1390, 1413, 11, 810, 181, 11, 91]
features_percent = [0.11165387299371947, 0.18143754361479414, 0.07048150732728542, 0.013956734124214934, 0.028611304954640614, 0.5519888346127007, 0.01465457083042568, 0.04256803907885555, 0.8304256803907886, 0.04884856943475227, 0.006978367062107467, 0.006978367062107467, 0.5931612002791347, 0.055826936496859735, 0.03558967201674808, 0.08443824145150035, 0.8792742498255408, 0.34263782274947663, 0.03558967201674808, 0.7264480111653873, 0.0, 0.04256803907885555, 0.034891835310537335, 0.0628053035589672, 0.04954640614096301, 0.034891835310537335, 0.18143754361479414, 0.0209351011863224, 0.25889741800418703, 0.8939288206559665, 0.0209351011863224, 0.9071877180739707, 0.03558967201674808, 0.8653175157013259, 0.07676203768318214, 0.0006978367062107466, 0.03558967201674808, 0.028611304954640614, 0.0, 0.7473831123517097, 0.8032100488485694, 0.0006978367062107466, 0.677599441730635, 0.0418702023726448, 0.028611304954640614, 0.013956734124214934, 0.013956734124214934, 0.0209351011863224, 0.021632937892533146, 0.03558967201674808, 0.028611304954640614, 0.01465457083042568, 0.013956734124214934, 0.007676203768318213, 0.027913468248429867, 0.9706908583391486, 0.034891835310537335, 0.007676203768318213, 0.4263782274947662, 0.08443824145150035, 0.021632937892533146, 0.7124912770411723, 0.11863224005582694, 0.07048150732728542, 0.0209351011863224, 0.021632937892533146, 0.0, 0.21004884856943476, 0.19539427773900908, 0.0209351011863224, 0.007676203768318213, 0.028611304954640614, 0.47522679692951847, 0.013956734124214934, 0.06978367062107467, 0.021632937892533146, 0.04256803907885555, 0.0, 0.7892533147243544, 0.8513607815771109, 0.07676203768318214, 0.11933007676203769, 0.13258897418004187, 0.01465457083042568, 0.8932309839497558, 0.0418702023726448, 0.9699930216329379, 0.9860432658757851, 0.007676203768318213, 0.5652477320307048, 0.12630844382414516, 0.007676203768318213, 0.06350314026517795]


In [159]:
name_list = ['iterations', 'nodes', 'nodes_percent', 'features', 'features_percent']
values_list = [iterations, nodes, nodes_percent, features, features_percent]

for i in range(len(values_list)):
    element = values_list[i]
    print(name_list[i])
    print('mean: ', np.mean(element))
    print('median: ', np.median(element))
    print('-'*100)

In [144]:
plt.hist(nodes, rwidth=0.8)
plt.title('Number of nodes in the explanations of Cora')

In [145]:
plt.hist(nodes_percent, rwidth=0.8)
plt.title('Percent of nodes in the explanations of Cora')

In [146]:
plt.hist(features, rwidth=0.8)
plt.title('Number of features in the explanations of Cora')

In [147]:
plt.hist(features_percent, rwidth=0.8)
plt.title('Percent of features in the explanations of Cora')

In [157]:
plt.hist(iterations, rwidth=0.8)
plt.title('Number of iterations of zorro in Cora')

##### PubMed

In [114]:
# tau: 0.3, initial
# time_taken for explaining:  1208.7117369174957
# time_taken for explaining + evaluation:  1226.966651916504

fidelity, node_sparsity_original, feature_sparsity = load_zorro_baseline('PubMed', 'gcn', 'zorro')
print('Num of nodes: ', len(fidelity))

In [118]:
len(node_sparsity_original)

In [116]:
feature_sparsity

In [119]:
# tau: 0.1, initial
# time_taken for explaining:  5286.470011472702
# time_taken for explaining + evaluation:  5304.453564167023

fidelity, node_sparsity_original, feature_sparsity = load_zorro_baseline('PubMed', 'gcn', 'zorro')
print('Num of nodes: ', len(fidelity))

In [160]:
iterations = [4, 15, 9, 80, 104, 7, 0, 0, 7, 2, 112, 4, 3, 18, 3, 12, 3, 2, 5, 0, 46, 0, 65, 3, 3, 4, 14, 3, 11, 4, 7, 3, 3, 0, 7, 72, 96, 0, 19, 10, 0, 4, 0, 7, 0, 121, 6, 2, 4, 124, 77, 16, 245, 4, 9, 68, 10, 0, 106, 10, 7, 5, 6, 5, 85, 3, 7, 6, 69, 0, 80, 3, 0, 0, 13, 4, 60, 0, 83, 5, 0, 3, 3, 7, 11, 4, 3, 30, 3, 36, 3, 24, 0, 0]
nodes = [5, 7, 11, 62, 114, 10, 0, 0, 1, 2, 154, 1, 2, 12, 2, 5, 3, 1, 3, 0, 56, 0, 84, 4, 3, 3, 20, 2, 12, 4, 4, 3, 2, 0, 5, 64, 98, 0, 26, 12, 0, 3, 0, 7, 0, 140, 8, 2, 3, 154, 56, 26, 388, 2, 10, 33, 12, 0, 132, 4, 5, 4, 8, 6, 85, 3, 5, 8, 36, 0, 78, 3, 0, 0, 6, 4, 52, 0, 82, 2, 0, 3, 3, 10, 12, 4, 1, 34, 4, 11, 3, 30, 0, 0]
nodes_percent = [0.14285714285714285, 0.047619047619047616, 0.2972972972972973, 0.8157894736842105, 1.0, 0.22727272727272727, 0.0, 0.0, 0.015625, 0.013157894736842105, 0.7333333333333333, 0.007692307692307693, 0.00338409475465313, 0.8, 0.006993006993006993, 0.13157894736842105, 0.01948051948051948, 0.00423728813559322, 0.016216216216216217, 0.0, 0.6086956521739131, 0.0, 0.8235294117647058, 0.1111111111111111, 0.012658227848101266, 0.011764705882352941, 0.273972602739726, 0.005988023952095809, 0.0736196319018405, 0.019704433497536946, 0.008869179600886918, 0.024193548387096774, 0.009389671361502348, 0.0, 0.050505050505050504, 0.6956521739130435, 0.7777777777777778, 0.0, 0.36619718309859156, 0.18461538461538463, 0.0, 0.018292682926829267, 0.0, 0.09090909090909091, 0.0, 0.8536585365853658, 0.0851063829787234, 0.007246376811594203, 0.07692307692307693, 0.5480427046263345, 0.6436781609195402, 0.12745098039215685, 0.8237791932059448, 0.0034423407917383822, 0.1111111111111111, 0.5409836065573771, 0.1111111111111111, 0.0, 0.6376811594202898, 1.0, 0.043478260869565216, 0.05263157894736842, 0.125, 0.42857142857142855, 0.6159420289855072, 0.12, 0.028409090909090908, 0.08080808080808081, 0.36, 0.0, 0.5131578947368421, 0.04285714285714286, 0.0, 0.0, 0.058823529411764705, 0.6666666666666666, 0.5252525252525253, 0.0, 0.31906614785992216, 0.004, 0.0, 0.016483516483516484, 0.0027881040892193307, 0.19230769230769232, 0.8571428571428571, 0.05555555555555555, 0.004329004329004329, 0.8947368421052632, 0.2857142857142857, 0.7333333333333333, 0.023622047244094488, 0.3488372093023256, 0.0, 0.0]
features = [10, 110, 21, 480, 461, 11, 0, 0, 60, 1, 341, 30, 11, 111, 11, 90, 10, 10, 30, 0, 171, 0, 221, 1, 10, 20, 31, 11, 41, 11, 41, 10, 11, 0, 40, 391, 461, 0, 51, 31, 0, 20, 0, 30, 0, 500, 11, 1, 20, 461, 480, 21, 500, 21, 31, 500, 31, 0, 391, 71, 40, 21, 11, 11, 420, 10, 40, 11, 500, 0, 401, 10, 0, 0, 91, 11, 331, 0, 411, 31, 0, 10, 10, 11, 41, 11, 20, 121, 1, 300, 10, 81, 0, 0]
features_percent = [0.02, 0.22, 0.042, 0.96, 0.922, 0.022, 0.0, 0.0, 0.12, 0.002, 0.682, 0.06, 0.022, 0.222, 0.022, 0.18, 0.02, 0.02, 0.06, 0.0, 0.342, 0.0, 0.442, 0.002, 0.02, 0.04, 0.062, 0.022, 0.082, 0.022, 0.082, 0.02, 0.022, 0.0, 0.08, 0.782, 0.922, 0.0, 0.102, 0.062, 0.0, 0.04, 0.0, 0.06, 0.0, 1.0, 0.022, 0.002, 0.04, 0.922, 0.96, 0.042, 1.0, 0.042, 0.062, 1.0, 0.062, 0.0, 0.782, 0.142, 0.08, 0.042, 0.022, 0.022, 0.84, 0.02, 0.08, 0.022, 1.0, 0.0, 0.802, 0.02, 0.0, 0.0, 0.182, 0.022, 0.662, 0.0, 0.822, 0.062, 0.0, 0.02, 0.02, 0.022, 0.082, 0.022, 0.04, 0.242, 0.002, 0.6, 0.02, 0.162, 0.0, 0.0]

In [161]:
name_list = ['iterations', 'nodes', 'nodes_percent', 'features', 'features_percent']
values_list = [iterations, nodes, nodes_percent, features, features_percent]

for i in range(len(values_list)):
    element = values_list[i]
    print(name_list[i])
    print('mean: ', np.mean(element))
    print('median: ', np.median(element))
    print('-'*100)

In [132]:
np.mean(features_percent)

In [162]:
plt.hist(nodes, rwidth=0.8)
plt.title('Number of nodes in the explanations of PubMed')

In [163]:
plt.hist(nodes_percent, rwidth=0.8)
plt.title('Percent of nodes in the explanations of PubMed')

In [164]:
plt.hist(features, rwidth=0.8)
plt.title('Number of features in the explanations of PubMed')

In [165]:
plt.hist(features_percent, rwidth=0.8)
plt.title('Percent of features in the explanations of PubMed')

In [166]:
plt.hist(iterations, rwidth=0.8)
plt.title('Number of iterations of zorro in PubMed')

### PGMExplainer

Only prodives node masks. Current computation is based on 100 perturbations.

#### Cora

In [124]:
# Full computation graph
# time_taken for explaining:  127.8276960849762
# time_taken for explaining + evaluation:  138.80583024024963

fidelity, node_sparsity_original = load_pgmexplainer('cora', 'gcn', 'pgmexplainer')
print('Num of nodes: ', len(fidelity))

In [117]:
# reduced, top_nodes: 50% of the largest number of neighbors
# time_taken for explaining:  121.6539032459259
# time_taken for explaining + evaluation:  133.09202003479004

fidelity, node_sparsity_original = load_pgmexplainer('cora', 'gcn', 'pgmexplainer')
print('Num of nodes: ', len(fidelity))

In [120]:
# reduced, top_nodes: 10 for each side
# time_taken for explaining:  104.52897024154663
# time_taken for explaining + evaluation:  115.85448598861694

fidelity, node_sparsity_original = load_pgmexplainer('cora', 'gcn', 'pgmexplainer')
print('Num of nodes: ', len(fidelity))

In [122]:
# reduced, top_nodes: 5 for each side
# time_taken for explaining:  89.68389654159546
# time_taken for explaining + evaluation:  100.00922393798828

fidelity, node_sparsity_original = load_pgmexplainer('cora', 'gcn', 'pgmexplainer')
print('Num of nodes: ', len(fidelity))

#### PubMed

In [136]:
# Full computation graph
# time_taken for explaining:  1833.5620889663696
# time_taken for explaining + evaluation:  1855.8837749958038
fidelity, node_sparsity_original = load_pgmexplainer('PubMed', 'gcn', 'pgmexplainer')
print('Num of nodes: ', len(fidelity))

In [138]:
# reduced, top_nodes: 10 for each side
# time_taken for explaining:  969.7029597759247
# time_taken for explaining + evaluation:  992.4953420162201
fidelity, node_sparsity_original = load_pgmexplainer('PubMed', 'gcn', 'pgmexplainer')
print('Num of nodes: ', len(fidelity))

In [137]:
# reduced, top_nodes: 5 for each side
# time_taken for explaining:  743.9129590988159
# time_taken for explaining + evaluation:  766.5984990596771
fidelity, node_sparsity_original = load_pgmexplainer('PubMed', 'gcn', 'pgmexplainer')
print('Num of nodes: ', len(fidelity))

### Zorro Baseline (Select top similar nodes, all features are selected.)

In [50]:
# 修改后，只加入增加fidelity的node，最大fidelity降低了，但sparsity也降低了
# more compact

# 需要visualize node mask的distribution


# tau: 0.05
# time_taken for explaining:  263.54108929634094
# time_taken for explaining + evaluation:  273.93209290504456
fidelity, node_sparsity_original = load_zorro_baseline('cora', 'gcn', 'zorro_baseline')
print('Num of nodes: ', len(fidelity))

In [51]:
node_mask_proportion_list = [0.6129032258064516, 0.14705882352941177, 0.525974025974026, 0.21052631578947367, 0.43333333333333335, 0.3076923076923077, 0.3181818181818182, 0.25, 0.15517241379310345, 0.36363636363636365, 0.3333333333333333, 0.02318840579710145, 0.6, 0.1346153846153846, 0.1724137931034483, 0.14655172413793102, 0.5555555555555556, 0.46153846153846156, 0.391304347826087, 0.3888888888888889, 0.043478260869565216, 0.35714285714285715, 0.38461538461538464, 0.2781065088757396, 0.3333333333333333, 0.4166666666666667, 0.6666666666666666, 0.21739130434782608, 0.4957983193277311, 0.3333333333333333, 0.5384615384615384, 0.07692307692307693, 0.21153846153846154, 0.5384615384615384, 0.1348314606741573, 0.2, 0.36363636363636365, 0.43243243243243246, 0.0, 0.5308641975308642, 0.37037037037037035, 0.5, 0.425531914893617, 0.16, 0.6122448979591837, 0.11764705882352941, 0.04477611940298507, 0.4444444444444444, 0.11764705882352941, 0.5609756097560976, 0.375, 0.391304347826087, 0.47619047619047616, 0.04941860465116279, 0.3125, 0.36363636363636365, 0.2962962962962963, 0.5, 0.16981132075471697, 0.2631578947368421, 0.37142857142857144, 0.13333333333333333, 0.075, 0.21311475409836064, 0.1643835616438356, 0.4426229508196721, 0.056338028169014086, 0.1780821917808219, 0.2777777777777778, 0.5, 0.23717948717948717, 0.6666666666666666, 0.654320987654321, 0.12790697674418605, 0.1610738255033557, 0.11538461538461539, 0.14743589743589744, 0.2857142857142857, 0.6666666666666666, 0.1875, 0.06467661691542288, 0.225, 0.15254237288135594, 0.3541666666666667, 0.09210526315789473, 0.23333333333333334, 0.31666666666666665, 0.16216216216216217, 0.15625, 0.08421052631578947, 0.4, 0.017543859649122806, 0.328125]

In [60]:
plt.hist(node_mask_proportion_list, rwidth=0.9)

In [56]:
pgm_pro_list = [0.3225806451612903, 0.14705882352941177, 0.5454545454545454, 0.47368421052631576, 0.6333333333333333, 0.38461538461538464, 0.6363636363636364, 0.575, 0.6206896551724138, 0.6363636363636364, 0.5833333333333334, 0.17681159420289855, 0.35, 0.40384615384615385, 0.47126436781609193, 0.3706896551724138, 0.6111111111111112, 0.23076923076923078, 0.21739130434782608, 0.1388888888888889, 0.5217391304347826, 0.7142857142857143, 0.3076923076923077, 0.07100591715976332, 0.41025641025641024, 0.5, 0.5333333333333333, 0.5217391304347826, 0.6890756302521008, 0.5185185185185185, 0.6153846153846154, 0.08791208791208792, 0.75, 0.41025641025641024, 0.19662921348314608, 0.45, 0.6363636363636364, 0.7837837837837838, 0.7777777777777778, 0.5061728395061729, 0.5555555555555556, 0.5, 0.425531914893617, 0.68, 0.6326530612244898, 0.19117647058823528, 0.6119402985074627, 0.6666666666666666, 0.17647058823529413, 0.3719512195121951, 0.625, 0.34782608695652173, 0.7619047619047619, 0.18895348837209303, 0.75, 0.5454545454545454, 0.5185185185185185, 0.8, 0.4528301886792453, 0.631578947368421, 0.8571428571428571, 0.4, 0.8, 0.6967213114754098, 0.0684931506849315, 0.5655737704918032, 0.09154929577464789, 0.6986301369863014, 0.5, 0.5, 0.07692307692307693, 0.3333333333333333, 0.6296296296296297, 0.5348837209302325, 0.2348993288590604, 0.4423076923076923, 0.6025641025641025, 0.42857142857142855, 0.3333333333333333, 0.6875, 0.13930348258706468, 0.625, 0.23728813559322035, 0.40625, 0.4868421052631579, 0.6, 0.35, 0.6486486486486487, 0.34375, 0.3263157894736842, 0.32, 0.08771929824561403, 0.421875]


In [61]:
plt.hist(pgm_pro_list, rwidth=0.9)

#### Cora

##### No Reduction

In [131]:
# tau: 0.15
# time_taken for explaining:  135.70069932937622
# time_taken for explaining + evaluation:  145.3336398601532
fidelity, node_sparsity_original = load_zorro_baseline('cora', 'gcn', 'zorro_baseline')
print('Num of nodes: ', len(fidelity))

In [132]:
# tau: 0.1
# time_taken for explaining:  177.11692929267883
# time_taken for explaining + evaluation:  187.08610606193542
fidelity, node_sparsity_original = load_zorro_baseline('cora', 'gcn', 'zorro_baseline')
print('Num of nodes: ', len(fidelity))

In [133]:
# tau: 0.05
# time_taken for explaining:  203.28461027145386
# time_taken for explaining + evaluation:  212.75743412971497
fidelity, node_sparsity_original = load_zorro_baseline('cora', 'gcn', 'zorro_baseline')
print('Num of nodes: ', len(fidelity))

##### Reduction

#### PubMed

##### No Reduction

In [144]:
# tau: 0.15
# time_taken for explaining:  205.91293597221375
# time_taken for explaining + evaluation:  224.90335392951965
fidelity, node_sparsity_original = load_zorro_baseline('PubMed', 'gcn', 'zorro_baseline')
print('Num of nodes: ', len(fidelity))

In [145]:
# tau: 0.1
# time_taken for explaining:  248.14503860473633
# time_taken for explaining + evaluation:  267.48149704933167
fidelity, node_sparsity_original = load_zorro_baseline('PubMed', 'gcn', 'zorro_baseline')
print('Num of nodes: ', len(fidelity))

In [146]:
# tau: 0.05
# time_taken for explaining:  336.1451451778412
# time_taken for explaining + evaluation:  356.6368088722229
fidelity, node_sparsity_original = load_zorro_baseline('PubMed', 'gcn', 'zorro_baseline')
print('Num of nodes: ', len(fidelity))

### GNNExplainer

#### PubMed

In [135]:
# time_taken for explaining:  1503.9144859313965
# time_taken for explaining + evaluation:  1542.3847279548645
fidelity = load_metrics('PubMed', 'gcn', 'gnnexplainer')
print('Num of nodes: ', len(fidelity))

#### PubMed

In [55]:
# Original neighbor, 30min to get the explanations for 100 samples
fidelity, node_sparsity_original = load_pgmexplainer('PubMed', 'gcn', 'pgmexplainer')
print('Num of nodes: ', len(fidelity))

In [57]:
# Reduced neighbor, 48min to get the explanations for 100 samples
fidelity, node_sparsity_original = load_pgmexplainer('PubMed', 'gcn', 'pgmexplainer')
print('Num of nodes: ', len(fidelity))

In [58]:
node_sparsity_original = node_sparsity_original[~np.isnan(node_sparsity_original)]
np.mean(node_sparsity_original)

### SoftZorro

1736.182s on Cora

In [62]:
metric_list = [{'link': (859, 1199), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.51, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.0}, {'link': (88, 2017), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.7, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (609, 1041), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.48, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (43, 963), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.77, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (718, 2059), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.59, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.6931472}, {'link': (1191, 2339), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.47, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.6931472}, {'link': (868, 1526), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.65, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.0}, {'link': (657, 1254), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.46, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.0}, {'link': (1416, 1926), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.62, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (108, 2157), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.49, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 1.0986123}, {'link': (867, 2522), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.73, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 1.0986123}, {'link': (796, 1358), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.4, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (16, 2444), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.48, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.0}, {'link': (1309, 2102), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.55, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (218, 1348), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.42, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.0}, {'link': (1810, 1815), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.45, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (697, 1080), 'true_label': 1.0, 'prediction_with_comp_subgraph': 0, 'fidelity': 0.51, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 1.609438}, {'link': (372, 2377), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.5, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.6931472}, {'link': (1046, 2387), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.49, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.6931472}, {'link': (946, 1966), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.55, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (807, 1956), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.94, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 1.0986123}, {'link': (868, 1844), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.67, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.6931472}, {'link': (1428, 1429), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.77, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.6931472}, {'link': (1070, 1167), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.52, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (277, 553), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.48, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.6931472}, {'link': (1451, 2564), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.65, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 1.0986123}, {'link': (1289, 1622), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.42, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.0}, {'link': (857, 1097), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.7, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 1.0986123}, {'link': (598, 1100), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.52, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (1484, 2383), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.62, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 1.0986123}, {'link': (80, 2049), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.46, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 1.0986123}, {'link': (1561, 1623), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.47, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (1079, 1218), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.52, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.0}, {'link': (312, 1610), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.53, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.6931472}, {'link': (1070, 1470), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.56, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (4, 1761), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.87, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.6931472}, {'link': (890, 1314), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.78, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 1.3862944}, {'link': (829, 1816), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.49, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 1.3862944}, {'link': (1085, 2110), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.97, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 1.609438}, {'link': (598, 1003), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.49, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (2281, 2405), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.65, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (385, 2484), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.85, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.0}, {'link': (162, 1667), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.51, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (1826, 1827), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.71, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.0}, {'link': (1478, 1902), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.54, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (1405, 1894), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.67, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (1914, 1918), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.51, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (2224, 2226), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.76, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 1.3862944}, {'link': (1952, 2204), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.57, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.6931472}, {'link': (1742, 1758), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.68, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (1896, 1897), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.64, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 1.0986123}, {'link': (2246, 2247), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.57, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 1.0986123}, {'link': (381, 2518), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.73, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 1.3862944}, {'link': (1358, 1733), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.38, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (1408, 1829), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.65, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.6931472}, {'link': (1202, 2376), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.6, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.6931472}, {'link': (728, 1296), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.52, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 1.0986123}, {'link': (2128, 2129), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.84, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 1.0986123}, {'link': (901, 2199), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.55, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (89, 1401), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.67, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 1.0986123}, {'link': (778, 1370), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.54, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (627, 2238), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.47, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (447, 1849), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.79, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.0}, {'link': (133, 452), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.64, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (69, 1920), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.63, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (226, 1701), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.55, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (155, 817), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.9, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (1120, 1583), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.58, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (1050, 2563), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.75, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.0}, {'link': (727, 2570), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.69, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.6931472}, {'link': (957, 1557), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.73, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (167, 1056), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.43, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.0}, {'link': (484, 1367), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.52, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (1224, 1526), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.41, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (598, 1299), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.43, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (1842, 2405), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.88, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.0}, {'link': (609, 2446), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.64, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (122, 2455), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.92, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.6931472}, {'link': (117, 2537), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.55, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (2227, 2228), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.49, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (306, 2086), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.44, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (175, 2388), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.58, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.0}, {'link': (218, 2119), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.51, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (910, 2085), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.66, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (598, 637), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.57, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (489, 2391), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.83, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (963, 1703), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.52, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (688, 1257), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.69, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': 0.0}, {'link': (240, 1692), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.75, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (45, 2667), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.7, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (518, 666), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.59, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (109, 306), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.82, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}, {'link': (2025, 2026), 'true_label': 1.0, 'prediction_with_comp_subgraph': 1, 'fidelity': 0.51, 'fidelity_with_edge_mask': None, 'feature_sparsity': np.nan, 'edge_sparsity': None, 'node_sparsity_converted': None, 'node_sparsity_original': np.nan}]


In [64]:
fidelity = np.array([r['fidelity'] for r in metric_list])
feature_sparsity = np.array([r['feature_sparsity'] for r in metric_list])
node_sparsity_original = np.array([r['node_sparsity_original'] for r in metric_list])

print('fidelity: ', calculate_mean_std(fidelity))
print('feature_sparsity: ', calculate_mean_std(feature_sparsity))
print('node_sparsity_original: ', calculate_mean_std(node_sparsity_original))

In [66]:
node_sparsity_original

In [ ]:
# PubMed
# fidelity_with_edge_mask using bernoulli_whole as the noise

In [13]:
fidelity = load_metrics('PubMed', 'gcn', 'gnnexplainer')
print('Num of nodes: ', len(fidelity))

In [14]:
fidelity = load_metrics('PubMed', 'gcn', 'ig')
print('Num of nodes: ', len(fidelity))

In [15]:
fidelity = load_metrics('PubMed', 'gcn', 'deconvolution')
print('Num of nodes: ', len(fidelity))

### GNNExplainer

#### PubMed

In [10]:
# use all features, only perturb the edge masks
fidelity = load_metrics('PubMed', 'gcn', 'gnnexplainer')
print('Num of nodes: ', len(fidelity))

#### Cora

Despite drastic changes in edge masks (different noise types), the influence on final RDT-Fideility is minor.
This indicates that edge masks may not be as important as feature masks.

While using all features and only perturb the edge masks, the edge-level fidelity still achieves 0.9667.

In [47]:
# use all features, only perturb the edge masks
fidelity = load_metrics('cora', 'gcn', 'gnnexplainer')
print('Num of nodes: ', len(fidelity))

In [46]:
# edge mask all 1s, using all edges equally, perturb the features
# edge weights all 1s, no edge noise
fidelity = load_metrics('cora', 'gcn', 'gnnexplainer')
print('Num of nodes: ', len(fidelity))

In [54]:
# bernoulli, computation 最新 1.25
fidelity = load_metrics('cora', 'gcn', 'gnnexplainer')
print('Num of nodes: ', len(fidelity))

In [52]:
# bernoulli, whole 最新 1.25
fidelity = load_metrics('cora', 'gcn', 'gnnexplainer')
print('Num of nodes: ', len(fidelity))

In [55]:
# kde, fitted on 1 - mask 最新 1.25
fidelity = load_metrics('cora', 'gcn', 'gnnexplainer')
print('Num of nodes: ', len(fidelity))

In [40]:
# kde, fitted on mask 最新 1.25
fidelity = load_metrics('cora', 'gcn', 'gnnexplainer')
print('Num of nodes: ', len(fidelity))

In [5]:
# 1.28 概率为1/2
fidelity = load_metrics('cora', 'gcn', 'gnnexplainer')
print('Num of nodes: ', len(fidelity))

In [56]:
# 高斯噪声
# import torch

# np.random.seed(12345)
# noise = np.random.normal(0, 0.5, size=(100, 96))
# noise

In [29]:
noise.shape

### IG

In [57]:
# kde, fitted on 1 - mask 最新 1.25
fidelity = load_metrics('cora', 'gcn', 'ig')
print('Num of nodes: ', len(fidelity))

In [58]:
# kde, fitted on mask 最新 1.25
fidelity = load_metrics('cora', 'gcn', 'ig')
print('Num of nodes: ', len(fidelity))

In [59]:
# bernoulli, whole 最新 1.25
fidelity = load_metrics('cora', 'gcn', 'ig')
print('Num of nodes: ', len(fidelity))

In [60]:
# bernoulli, computation 最新 1.25
fidelity = load_metrics('cora', 'gcn', 'ig')
print('Num of nodes: ', len(fidelity))

In [6]:
# 1.28 概率为1/2
fidelity = load_metrics('cora', 'gcn', 'ig')
print('Num of nodes: ', len(fidelity))

### Deconvolution

In [61]:
# bernoulli, computation 最新 1.25
fidelity = load_metrics('cora', 'gcn', 'deconvolution')
print('Num of nodes: ', len(fidelity))

In [62]:
# bernoulli, whole 最新 1.25
fidelity = load_metrics('cora', 'gcn', 'deconvolution')
print('Num of nodes: ', len(fidelity))

In [63]:
# kde, fitted on mask 最新 1.25
fidelity = load_metrics('cora', 'gcn', 'deconvolution')
print('Num of nodes: ', len(fidelity))

In [64]:
# kde, fitted on 1 - mask 最新 1.25
fidelity = load_metrics('cora', 'gcn', 'deconvolution')
print('Num of nodes: ', len(fidelity))

In [7]:
# 1.28 概率为1/2
fidelity = load_metrics('cora', 'gcn', 'deconvolution')
print('Num of nodes: ', len(fidelity))

### Empty

All same since we use fixed value as random seed.

Empty explanation: edge mask and feature mask all 0s.

1. Original RDT-Fidelity:
   pertrubed input: only use random values for all features
2. RDT-Fidelity with edge mask:
   perturbed edge importance: only use random noise as feature importance

In [65]:
# kde, fitted on 1 - mask 最新 1.25
fidelity = load_metrics('cora', 'gcn', 'empty')
print('Num of nodes: ', len(fidelity))

In [8]:
# kde, fitted on 1 - mask 最新 1.25
fidelity = load_metrics('cora', 'gcn', 'empty')
print('Num of nodes: ', len(fidelity))

In [5]:
# # bernoulli, computation
# fidelity = load_metrics('cora', 'gcn', 'gnnexplainer')
# print('Num of nodes: ', len(fidelity_list))

In [4]:
# bernoulli, whole
fidelity = load_metrics('cora', 'gcn', 'gnnexplainer')
print('Num of nodes: ', len(fidelity))

In [6]:
# kde, fitted on mask
fidelity = load_metrics('cora', 'gcn', 'gnnexplainer')
print('Num of nodes: ', len(fidelity))

In [ ]:
# kde, fitted on 1-mask


In [129]:
fidelity = load_metrics('cora', 'gcn', 'ig')
print('Num of nodes: ', len(fidelity_list))

In [130]:
fidelity = load_metrics('cora', 'gcn', 'deconvolution')
print('Num of nodes: ', len(fidelity_list))

In [131]:
# cora, gcn, gnnexplainer
fidelity = load_metrics('cora', 'gcn', 'empty')
print('Num of nodes: ', len(fidelity_list))

In [48]:
# cora_gcn_gnnexplainer
fidelity_cora_gcn_gnnexplainer, validity_cora_gcn_gnnexplainer, \
feature_sparsity_cora_gcn_gnnexplainer, node_sparsity_cora_gcn_gnnexplainer = load_metrics('cora', 'gcn', 'gnnexplainer')
print('Num of nodes: ', len(fidelity_cora_gcn_gnnexplainer))

In [50]:
# cora_gcn_ig
fidelity_cora_gcn_ig, validity_cora_gcn_ig, \
feature_sparsity_cora_gcn_ig, node_sparsity_cora_gcn_ig = load_metrics('cora', 'gcn', 'ig')
print('Num of nodes: ', len(fidelity_cora_gcn_ig))

In [51]:
# cora_gcn_deconvolution
fidelity_cora_gcn_deconvolution, validity_cora_gcn_deconvolution, \
feature_sparsity_cora_gcn_deconvolution, node_sparsity_cora_gcn_deconvolution = load_metrics('cora', 'gcn', 'deconvolution')
print('Num of nodes: ', len(fidelity_cora_gcn_deconvolution))

In [54]:
# cora_gcn_grad
fidelity_cora_gcn_grad, validity_cora_gcn_grad, \
feature_sparsity_cora_gcn_grad, node_sparsity_cora_gcn_grad = load_metrics('cora', 'gcn', 'grad')
print('Num of nodes: ', len(fidelity_cora_gcn_grad))

In [63]:
# cora_gcn_empty
fidelity_cora_gcn_empty, validity_cora_gcn_empty = load_fidelity_validity('cora', 'gcn', 'empty')
print('Num of nodes: ', len(fidelity_cora_gcn_empty))

In [53]:
# cora_gin_gnnexplainer
fidelity_cora_gin_gnnexplainer, validity_cora_gin_gnnexplainer, \
feature_sparsity_cora_gin_gnnexplainer, node_sparsity_cora_gin_gnnexplainer = load_metrics('cora', 'gin', 'gnnexplainer')

In [70]:
# RDT-Fidelity vis

fig, ax = plt.subplots(figsize=(8, 4))

tot_len = len(fidelity_cora_gcn_gnnexplainer)

data = pd.DataFrame({
    'Group': np.repeat(['GNNExplainer', 'IG', 'Deconvolution', 'Grad', 'Empty'], tot_len),
    'Value': np.concatenate([fidelity_cora_gcn_gnnexplainer[:tot_len], 
                             fidelity_cora_gcn_ig[:tot_len],
                             fidelity_cora_gcn_deconvolution[:tot_len],
                             fidelity_cora_gcn_grad[:tot_len],
                             fidelity_cora_gcn_empty[:tot_len]])
})

fig, ax = plt.subplots(figsize=(8, 4))
sns.violinplot(x='Group', y='Value', data=data)
ax.set_ylim([0, 1])
ax.set_yticks([0, 0.5, 1])
plt.title('RDT-fidelity Distribution of GCN on Cora')
plt.xlabel('Group')
plt.ylabel('Value')
plt.show()

In [65]:
fidelity_cora_gcn_gnnexplainer

In [ ]:
len(fidelity_cora_gin_gnnexplainer)

In [11]:
tot_len = min(len(fidelity_cora_gcn_gnnexplainer), len(fidelity_cora_gin_gnnexplainer))
tot_len

In [12]:
data = pd.DataFrame({
    'Group': np.repeat(['RDT-fidelity, GCN', 'RDT-fidelity, GIN', 'validity, GCN', 'validity, GIN'], tot_len),
    'Value': np.concatenate([fidelity_cora_gcn_gnnexplainer[:tot_len], 
                             fidelity_cora_gin_gnnexplainer[:tot_len],
                             validity_cora_gcn_gnnexplainer[:tot_len],
                             validity_cora_gin_gnnexplainer[:tot_len]])
})

fig, ax = plt.subplots(figsize=(8, 4))
sns.violinplot(x='Group', y='Value', data=data)
plt.title('cora_gnnexplainer, RDT-fidelity and validity')
plt.xlabel('Group')
plt.ylabel('Value')
plt.show()

In [145]:
data = pd.DataFrame({
    'Group': np.repeat(['feature_sparsity, GCN', 'feature_sparsity, GIN', 'node_sparsity, GCN', 'node_sparsity, GIN'], tot_len),
    'Value': np.concatenate([feature_sparsity_cora_gcn_gnnexplainer[:tot_len], 
                             feature_sparsity_cora_gin_gnnexplainer[:tot_len],
                             node_sparsity_cora_gcn_gnnexplainer[:tot_len],
                             node_sparsity_cora_gin_gnnexplainer[:tot_len]])
})

fig, ax = plt.subplots(figsize=(8, 4))
sns.violinplot(x='Group', y='Value', data=data)
plt.title('cora_gnnexplainer, feature_sparsity and node_sparsity')
plt.xlabel('Group')
plt.ylabel('Value')
plt.show()

In [163]:
num_nodes = len(node_sparsity_cora_gcn_gnnexplainer)

data = pd.DataFrame({
    'Group': np.repeat(['RDT-fidelity', 'validity', 'feature_sparsity', 'node_sparsity'], num_nodes),
    'Value': np.concatenate([fidelity_cora_gcn_gnnexplainer, 
                             validity_cora_gcn_gnnexplainer,
                             feature_sparsity_cora_gcn_gnnexplainer, 
                             node_sparsity_cora_gcn_gnnexplainer])
})

# Create a violin plot
sns.violinplot(x='Group', y='Value', data=data)
plt.title('cora_gcn_gnnexplainer')
plt.xlabel('Group')
plt.ylabel('Value')

# Show the plot
plt.show()

In [149]:
num_nodes = len(node_sparsity_cora_gin_gnnexplainer)

data = pd.DataFrame({
    'Group': np.repeat(['RDT-fidelity', 'validity', 'feature_sparsity', 'node_sparsity'], num_nodes),
    'Value': np.concatenate([fidelity_cora_gin_gnnexplainer, 
                             validity_cora_gin_gnnexplainer,
                             feature_sparsity_cora_gin_gnnexplainer, 
                             node_sparsity_cora_gin_gnnexplainer])
})

# Create a violin plot
sns.violinplot(x='Group', y='Value', data=data)
plt.title('cora_gin_gnnexplainer')
plt.xlabel('Group')
plt.ylabel('Value')

# Show the plot
plt.show()

In [135]:
# plt.hist(fidelity_list, bins=20)  # Adjust the number of bins as needed
# plt.title("Histogram")
# plt.xlabel("Value")
# plt.ylabel("Frequency")
# plt.show()

In [136]:
# plt.boxplot(fidelity_list)
# plt.title("Box Plot of Data Distribution")
# plt.ylabel("Value")
# plt.show()

## 2. Deletion Curve

In [71]:
from metrics import linear_area_score as area_score
from utils import normalize_bounds, load_curves

In [97]:
tot_range = 3000
graph_model = 'cora'
model_name = 'gcn'
  # ig, gnnexplainer, deconvolution
decoder = 'inner'
# ['vgae', 'gin', 'sage', 'gcn']
for model_name in ['gcn']:
    print(model_name, len(os.listdir(f"../outputs/{graph_model}/{model_name}/{explainer}/curves/")))

In [102]:
explainer = 'gnnexplainer'
edge_deletions = load_curves(graph_model, model_name, explainer, 'edge_deletion', tot_range, target=-1, seed='0')
gcn_edge_scores_gnnexplainer = np.array([area_score(d) for d in edge_deletions])
feat_deletions = load_curves(graph_model, model_name, explainer, 'feature_deletion', tot_range, target=-1, seed='0')
gcn_feat_scores_gnnexplainer = np.array([area_score(d) for d in feat_deletions])

In [98]:
explainer = 'ig'
edge_deletions = load_curves(graph_model, model_name, explainer, 'edge_deletion', tot_range, target=-1, seed='0')
gcn_edge_scores_ig = np.array([area_score(d) for d in edge_deletions])
feat_deletions = load_curves(graph_model, model_name, explainer, 'feature_deletion', tot_range, target=-1, seed='0')
gcn_feat_scores_ig = np.array([area_score(d) for d in feat_deletions])

In [100]:
explainer = 'deconvolution'
edge_deletions = load_curves(graph_model, model_name, explainer, 'edge_deletion', tot_range, target=-1, seed='0')
gcn_edge_scores_deconvolution = np.array([area_score(d) for d in edge_deletions])
feat_deletions = load_curves(graph_model, model_name, explainer, 'feature_deletion', tot_range, target=-1, seed='0')
gcn_feat_scores_deconvolution = np.array([area_score(d) for d in feat_deletions])

In [115]:
# Edge Deletion, GCN, Cora
tot_range = len(gcn_edge_scores_gnnexplainer)
df = pd.DataFrame(gcn_edge_scores_gnnexplainer[:tot_range], columns=['GNNExplainer'])
df['IG'] = gcn_edge_scores_ig[:tot_range]
df['Deconvolution'] = gcn_edge_scores_deconvolution[:tot_range]

fig, ax = plt.subplots(figsize=(6, 4))
sns.violinplot(df)
ax.axhline(0., c='black', ls='--')
ax.set_ylim([-1, 1])
ax.set_yticks([-1, 0, 1])
sns.despine()
plt.title('Edge Deletion of GCN on Cora')
plt.tight_layout()

In [116]:
# Feature Deletion, GCN, Cora
tot_range = len(gcn_feat_scores_gnnexplainer)
df = pd.DataFrame(gcn_feat_scores_gnnexplainer[:tot_range], columns=['GNNExplainer'])
df['IG'] = gcn_feat_scores_ig[:tot_range]
df['Deconvolution'] = gcn_feat_scores_deconvolution[:tot_range]

fig, ax = plt.subplots(figsize=(6, 4))
sns.violinplot(df)
ax.axhline(0., c='black', ls='--')
ax.set_ylim([-1, 1])
ax.set_yticks([-1, 0, 1])
sns.despine()
plt.title('Feature Deletion of GCN on Cora')
plt.tight_layout()

In [ ]:
# Edge Deletion, GCN, Cora
tot_range = len(gcn_feat_scores_gnnexplainer)
df = pd.DataFrame(gcn_edge_scores_gnnexplainer[:tot_range], columns=['gcn_edge'])
df['gcn_feat'] = gcn_feat_scores_gnnexplainer[:tot_range]
# df['gin_edge'] = gin_edge_scores[:tot_range]
# df['gin_feat'] = gin_feat_scores[:tot_range]

# df['sage_edge'] = sage_edge_scores[:tot_range]
# df['sage_feat'] = sage_feat_scores[:tot_range]
# df['vgae_feat'] = vgae_feat_scores[:tot_range]
# df['vgae_edge'] = vgae_edge_scores[:tot_range]

df.columns = ['GCN Edges Deletion', 'GCN Features Deletion']
# df.columns = ['GCN Edges Deletion', 'GCN Features Deletion', 'GIN Edges Deletion', 'GIN Features Deletion']

In [80]:
# model_name = 'gin'

# edge_deletions = load_curves(graph_model, model_name, explainer, 'edge_deletion', tot_range, target=-1, seed='0')
# gin_edge_scores = np.array([area_score(d) for d in edge_deletions])

# feat_deletions = load_curves(graph_model, model_name, explainer, 'feature_deletion', tot_range, target=-1, seed='0')
# gin_feat_scores = np.array([area_score(d) for d in feat_deletions])

In [169]:
tot_range = min(len(gcn_edge_scores), len(gin_edge_scores))
tot_range = len(gcn_edge_scores)
df = pd.DataFrame(gcn_edge_scores[:tot_range], columns=['gcn_edge'])
df['gcn_feat'] = gcn_feat_scores[:tot_range]
# df['gin_edge'] = gin_edge_scores[:tot_range]
# df['gin_feat'] = gin_feat_scores[:tot_range]

# df['sage_edge'] = sage_edge_scores[:tot_range]
# df['sage_feat'] = sage_feat_scores[:tot_range]
# df['vgae_feat'] = vgae_feat_scores[:tot_range]
# df['vgae_edge'] = vgae_edge_scores[:tot_range]

# df.columns = ['VGAE Edges', 'GIN Edges', 'SAGE Edges', 'GCN Edges', 'VGAE Features', 'GIN Features' , 'SAGE Features' , 'GCN Features']
df.columns = ['GCN Edges Deletion', 'GCN Features Deletion']
# df.columns = ['GCN Edges Deletion', 'GCN Features Deletion', 'GIN Edges Deletion', 'GIN Features Deletion']

In [170]:
fig, ax = plt.subplots(figsize=(8, 4))
sns.boxplot(df)
ax.axhline(0., c='black', ls='--')
ax.set_ylim([-1, 1])
ax.set_yticks([-1, 0, 1])
sns.despine()
plt.tight_layout()
# plt.savefig(f'../paper/figs/cora_area_score_{explainer}.png')

In [171]:
fig, ax = plt.subplots(figsize=(8, 4))
sns.violinplot(df)
ax.axhline(0., c='black', ls='--')
ax.set_ylim([-1, 1])
ax.set_yticks([-1, 0, 1])
sns.despine()
plt.tight_layout()
# plt.savefig(f'../paper/figs/cora_area_score_{explainer}.png')

In [172]:
print('gcn_edge_scores: ', calculate_mean_std(gcn_edge_scores))
print('gcn_feat_scores: ', calculate_mean_std(gcn_feat_scores))

In [173]:
print('gin_edge_scores: ', calculate_mean_std(gin_edge_scores))
print('gin_feat_scores: ', calculate_mean_std(gin_feat_scores))

In [113]:
gcn_edge_scores

# With Ground Truth

In [34]:
explainer = 'gnnexplainer'
graph = 'ws'

In [35]:
data = []
# ['vgae', 'gcn', 'gin', 'sage']
for model in ['gcn']:
    seed = 0
    path = f"../outputs/{graph}/{model}/{explainer}/"
    for filename in os.listdir(path):
        if not filename.startswith('.') and not filename in ['old', 'masks']:
            if filename.split('_')[-3] == 'inner':
                df = pd.read_csv(osp.join(path, filename), index_col=0)
                df['model'] = model.upper()
                data.append(df)

In [36]:
data = pd.concat(data)

In [37]:
model_perfomance = (data[['test_auc', 'model', 'seed']]
 .groupby(['model', 'seed'])
 .apply(lambda x: x.iloc[0])
 .test_auc
 .unstack('model')
)

In [38]:
if graph == 'ws':
    data = data[(data.source_node-data.target_node).abs()<(data.k/2+1)].copy()
if graph == 'sbm':
    data = data[(data.source_block_label==data.target_block_label)].copy()

In [39]:
data.columns

In [40]:
data.groupby(['seed', 'model']).size()

In [41]:
tpr = data[['tpr', 'model']].copy()
tpr.columns = ['value', 'model']
tpr['metric'] = 'sensitivity'
tnr = data[['tnr', 'model']].copy()
tnr.columns = ['value', 'model']
tnr['metric'] = 'specificity'
sb_data = pd.concat([tpr, tnr])

In [42]:
fig, ax = plt.subplots(figsize=(6, 3))
sns.violinplot(sb_data, x='model', y='value', hue='metric', inner='box', split=False)
ax.set_ylim([0, 1])
ax.set_yticks([0, 0.5, 1.])
ax.set_xlabel(None)
ax.set_ylabel(None)
sns.despine()
plt.tight_layout()
# plt.savefig(f'../paper/figs/{graph}_model_comparison_tpr_tnr_{explainer}.png')

In [43]:
tpr

In [44]:
print(np.mean(tpr.value))
print(np.std(tpr.value))

In [45]:
tnr

In [46]:
print(np.mean(tnr.value))
print(np.std(tnr.value))